In [1]:
# This code follows is a modified version of the tutorial from ligo.skymap
from astropy.coordinates import SkyCoord
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import crossmatch

/opt/miniconda3/envs/tilepyenv_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Skymap 
url = 'https://gracedb.ligo.org/api/superevents/S240615dg/files/Bilby.offline0.multiorder.fits'
skymap = read_sky_map(url, moc=True)

In [6]:
# Galaxy catalog
from tilepy.include.PointingTools import LoadGalaxies 

In [ ]:
catalog = '../../src/tilepy/dataset/Gladeplus.h5'
cat = LoadGalaxies(catalog)

Loading galaxy catalogue from ../../tilepy/dataset/Gladeplus.h5


In [11]:
cat

no_GLADE,RAJ2000,DEJ2000,Dist,Bmag
uint32,float64,float64,float32,float32
1078404,209.660614,28.544758,307.755,17.04
1078382,150.492096,27.61278,390.2103,17.033
1078358,171.639633,26.348652,216.03331,16.671
1078355,155.931229,26.182703,293.52972,16.539
1078350,160.374588,25.971264,356.17014,16.558
1078319,181.437912,24.259342,234.59818,16.125
1078292,162.406204,22.255919,226.93306,16.213
1078251,218.096588,19.566315,243.14256,17.153
1078114,159.47876,10.863852,433.2221,16.338


In [14]:
from astropy import units as u
coordinates = SkyCoord(cat['RAJ2000']*u.deg, cat['DEJ2000']*u.deg, cat['Dist']*u.Mpc)

In [15]:
result = crossmatch(skymap, coordinates)


In [19]:
#print(cat[result.searched_prob_vol < 0.9])
result

CrossmatchResult(searched_area=array([32739.1665124 , 36996.06388091, 17591.59480363, ...,
        2196.43146952,  2196.43146952,  2196.43146952]), searched_prob=array([1., 1., 1., ..., 1., 1., 1.]), offset=array([103.14481975,  99.30071145, 106.51358524, ...,  89.39273311,
        89.06493348,  89.03025649]), searched_modes=nan, contour_areas=[], area_probs=[], contour_modes=nan, searched_prob_dist=array([3.14463689e-06, 1.36309478e-05, 4.94113982e-07, ...,
       3.79554302e-05, 2.59884814e-08, 1.16913326e-05]), contour_dists=[], searched_vol=array([1.38263860e+12, 1.11437678e+12, 2.45408945e+12, ...,
       3.13224463e+11, 3.13231050e+11, 3.13224459e+11]), searched_prob_vol=array([1.00000331, 1.00000331, 1.00000331, ..., 1.00000331, 1.00000331,
       1.00000331]), contour_vols=[], probdensity=array([0., 0., 0., ..., 0., 0., 0.]), probdensity_vol=array([0., 0., 0., ..., 0., 0., 0.]))

In [24]:
from tilepy.include.PointingTools import LoadHealpixMap 
prob, distmu, distsigma, distnorm, detectors, fits_id, thisDistance, thisDistanceErr = LoadHealpixMap(url)
npix = len(prob)


Loading HEALPix map from file: https://gracedb.ligo.org/api/superevents/S240615dg/files/Bilby.offline0.multiorder.fits
Event has triggered  H1,L1,V1  => distance = 1420.02  +- 235.56  Mpc


In [23]:
import healpy as hp
nside = hp.npix2nside(npix)

In [26]:
from tilepy.include.PointingTools import CorrelateGalaxies_LVC
Gals, sum_dP_dV = CorrelateGalaxies_LVC(prob, distmu, distsigma, distnorm, cat, 'True', 0.002)